# AMPL Christmas Model created by ChatGPT
[![christmas.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/miscellaneous/christmas.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/miscellaneous/christmas.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/miscellaneous/christmas.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/miscellaneous/christmas.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/authors/fdabrandao/miscellaneous/christmas.ipynb)

Description: Christmas model generated by ChatGPT

Tags: christmas, amplpy

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

Model author: [ChatGPT](https://chat.openai.com/)

In [8]:
# Install dependencies
!pip install -q amplpy pandas

In [9]:
# Google Colab & Kaggle integration
MODULES=['ampl', 'highs']
from amplpy import tools
ampl = tools.ampl_notebook(modules=MODULES, globals_=globals()) # instantiate AMPL object and register magics

# Christmas model

**Description and model copied directly from [ChatGPT](https://chat.openai.com/) which kindly wrote this model for us**:

This model defines a set of people, a set of gifts, and the happiness level and cost of each gift. The decision variables `x[p,g]` represent whether each person receives each gift. The objective is to maximize the total happiness, subject to the budget constraint that the total cost of the gifts must be less than or equal to the budget, and the constraint that each person can only receive one gift. This model can be used to find the optimal gift-giving strategy using a linear programming solver.

In [10]:
%%ampl_eval
# define the set of people
set PEOPLE;

# define the set of gifts
set GIFTS;

# define the cost of each gift
param cost{GIFTS};

# define the happiness level for each person receiving each gift
param happiness{PEOPLE, GIFTS};

# define the budget for gift-giving
param budget;

# define the decision variables:
# x[p,g] is 1 if person p receives gift g, 0 otherwise
var x{PEOPLE, GIFTS} binary;

# the objective is to maximize total happiness
maximize total_happiness: sum{p in PEOPLE, g in GIFTS} happiness[p,g] * x[p,g];

# subject to the budget constraint:
# the total cost of the gifts must be less than or equal to the budget
subject to budget_constraint: sum{p in PEOPLE, g in GIFTS} cost[g] * x[p,g] <= budget;

# each person can only receive one gift
subject to one_gift_per_person{p in PEOPLE}: sum{g in GIFTS} x[p,g] = 1;

# Load data directly from python data structures

In [11]:
import pandas as pd
happiness_df = pd.DataFrame({
        "Book": [3, 2, 5, 1, 4],
        "Toy": [5, 2, 4, 3, 1],
        "Chocolate": [1, 3, 4, 5, 2],
        "Wine": [2, 5, 3, 4, 1],
        "Flowers": [4, 3, 1, 2, 5]
    }, index=["Alice", "Bob", "Carol", "Dave", "Eve"]
)

In [12]:
ampl.set["PEOPLE"] = ["Alice", "Bob", "Carol", "Dave", "Eve"]
ampl.set["GIFTS"] = ["Book", "Toy", "Chocolate", "Wine", "Flowers"]
ampl.param["cost"] = {"Book": 10, "Toy": 20, "Chocolate": 5, "Wine": 15, "Flowers":  7}
ampl.param["budget"] = 50
ampl.param["happiness"] = happiness_df.transpose().unstack()

# Solve with HiGHS

In [13]:
ampl.option["solver"] = "highs"
ampl.solve()

HiGHS 1.3.0: HiGHS 1.3.0: optimal solution; objective 24
1 branching nodes


# Retrieve solution

In [14]:
solution = ampl.get_data("{p in PEOPLE, g in GIFTS: x[p, g] > 0} x[p, g]").to_dict()
print("To maximize the total happiness:")
for person, gift in solution:
    print(f"- give {gift} to {person}.")

To maximize the total happiness:
- give Flowers to Alice.
- give Wine to Bob.
- give Book to Carol.
- give Chocolate to Dave.
- give Flowers to Eve.
